In [1]:
import numpy as np
from collections import Counter

In [2]:
# rerwite decission tree Algorithm with an option of selecting a random number of features

class node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    @property    
    def is_leaf_node(self):
        return self.value is not None

In [3]:
class DecisionTree:
    def __init__(self, min_sample_split=2, max_depth=100, n_random_features=None, mode='gini'):
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_random_features = n_random_features
        self.mode = mode
        
        self.root = None


    def fit(self, X, y):
        self.n_random_features = X.shape[1] if not self.n_random_features else min(self.n_random_features, X.shape[1])
        self.root = self._build_tree(X, y)


    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])


    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        if(depth >= self.max_depth or n_samples < self.min_sample_split or n_labels == 1):
            leaf_value = self._most_common_class(y)
            return node(value=leaf_value)

        features_indices = np.random.choice(n_features, self.n_random_features, replace=False)

        best_split_feature, best_split_threshold = self._best_split(X, y, features_indices)

        left_indices, right_indices = self._split(X[:, best_split_feature], best_split_threshold)

        left = self._build_tree(X[left_indices, :], y[left_indices], depth+1)
        right = self._build_tree(X[right_indices, :], y[right_indices], depth+1)

        return node(best_split_feature, best_split_threshold, left, right)


    def _best_split(self, X, y, features_indices):
        best_gain = -float('inf')
        split_index, split_threshold = None, None

        for index in features_indices:
            feature_column = X[:, index]
            thresholds = np.unique(feature_column)
            for threshold in thresholds:
                gain = self._information_gain(feature_column, threshold, y, self.mode)

                if gain > best_gain:
                    best_gain = gain
                    split_index = index
                    split_threshold = threshold

        return split_index, split_threshold


    def _information_gain(self, feature_column, threshold, y, mode):
        left_split_indices, right_split_indices = self._split(feature_column, threshold)

        if len(left_split_indices) == 0 or len(right_split_indices) == 0:
            return 0

        n_elements_left, n_elements_right = len(left_split_indices), len(right_split_indices)
        n_total_elements = len(y)

        left_weight = n_elements_left / n_total_elements
        right_weight = n_elements_right / n_total_elements

        if mode == 'entropy':
            parent_entropy = self._entropy(y)
            entropy_left, entropy_right = self._entropy(y[left_split_indices]), self._entropy(y[right_split_indices])
            child_entropy = left_weight*entropy_left + right_weight*entropy_right
            info_gain = parent_entropy - child_entropy

            return info_gain

        parent_gini = self._gini(y)
        gini_left, gini_right = self._gini(y[left_split_indices]), self._gini(y[right_split_indices])
        child_gini = left_weight*gini_left + right_weight*gini_right
        info_gain = parent_gini - child_gini

        return info_gain


    def _split(self, feature_column, threshold):
        left_split_indices = np.argwhere(feature_column <= threshold).flatten()
        right_split_indices = np.argwhere(feature_column > threshold).flatten()
        return left_split_indices, right_split_indices

    
    def _most_common_class(self, y):
        counter = Counter(y)
        common_label = counter.most_common(1)[0][0]
        return common_label
    

    def _entropy(self, y):
        histogram = np.bincount(y)
        classes_probs = histogram / len(y)
        return -np.sum([prob*np.log2(prob) for prob in classes_probs if prob > 0])


    def _gini(self, y):
        histogram = np.bincount(y)
        classes_probs = histogram / len(y)
        return 1 - np.sum(classes_probs**2)

    
    def _traverse_tree(self, x, node):
        if node.is_leaf_node:
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [4]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [5]:
dataset = datasets.load_breast_cancer()
X, y = dataset.data, dataset.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [6]:
def accuracy(original, predicted):
    return np.sum(original == predicted) / len(original)

In [7]:
clf = DecisionTree(max_depth=5)

In [8]:
clf.fit(X_train, y_train)

In [9]:
y_predicted = clf.predict(X_test)

In [10]:
acc = accuracy(y_test, y_predicted)

In [11]:
print(f'DecisionTree model accuracy = {acc*100:.1f}%')

DecisionTree model accuracy = 93.9%


In [12]:
# print('customized_DecisionTree.ipynb')